In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import copy
from collections import Counter

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
code_path = "/content/gdrive/MyDrive/cse512hw5/alice_release_new/"
os.chdir(code_path)

In [ ]:
#This was the process used to load and clean the corpus, and produce the corrupted corpus.
#You don't need to do anything here.
"""
corpus = []
f = open('alice_in_wonderland.txt','r')
while(1):
    line =  f.readline()
    if len(line) == 0: break
    corpus.extend(line.split())
        
f.close()


def clean_word(word):
    word = word.lower().strip()
    for punctuation in ['*','"',"'",'.',',','-','?','!',';',':','—','(',')','[',']']:
        
        word = ''.join(word.split(punctuation))
    
    return word

corpus = [clean_word(word) for word in corpus]
corpus = [word for word in corpus if len(word) > 0]

corrupted_corpus = copy.deepcopy(corpus)

p = .25
alphabet = 'abcdefghijklmnopqrstuvwxyz'
for k in xrange(len(corrupted_corpus)):
    word = corrupted_corpus[k]
    if len(word) < 2: continue
    if np.random.rand() < p:
        if np.random.randn() < 0:
            swap = np.random.choice(range(len(word)), size=2, replace = False)
            swap = np.sort(swap)
            word = ''.join([word[:swap[0]], word[swap[1]], word[swap[0]+1:swap[1]], word[swap[0]], word[swap[1]+1:]])
        else:
            
            replace = np.random.choice(range(len(word)), size=1, replace = False)[0]
            replace_with = alphabet[np.random.choice(range(len(alphabet)),size=1)[0]]
            word = ''.join([word[:replace], replace_with, word[replace+1:]])
        
        corrupted_corpus[k] = word

pickle.dump({'corpus':corpus,'corrupted_corpus':corrupted_corpus},open('alice_spelling.pkl','wb'))
"""

'\ncorpus = []\nf = open(\'alice_in_wonderland.txt\',\'r\')\nwhile(1):\n    line =  f.readline()\n    if len(line) == 0: break\n    corpus.extend(line.split())\n        \nf.close()\n\n\ndef clean_word(word):\n    word = word.lower().strip()\n    for punctuation in [\'*\',\'"\',"\'",\'.\',\',\',\'-\',\'?\',\'!\',\';\',\':\',\'—\',\'(\',\')\',\'[\',\']\']:\n        \n        word = \'\'.join(word.split(punctuation))\n    \n    return word\n\ncorpus = [clean_word(word) for word in corpus]\ncorpus = [word for word in corpus if len(word) > 0]\n\ncorrupted_corpus = copy.deepcopy(corpus)\n\np = .25\nalphabet = \'abcdefghijklmnopqrstuvwxyz\'\nfor k in xrange(len(corrupted_corpus)):\n    word = corrupted_corpus[k]\n    if len(word) < 2: continue\n    if np.random.rand() < p:\n        if np.random.randn() < 0:\n            swap = np.random.choice(range(len(word)), size=2, replace = False)\n            swap = np.sort(swap)\n            word = \'\'.join([word[:swap[0]], word[swap[1]], word[swap[0]

In [ ]:
#Take a look at how the data looks, and let's make some helper functions.
data = pickle.load(open('alice_spelling.pkl','rb'), encoding='utf-8')
vocab = np.unique(data['corpus'])
V = len(vocab)
print(V)

## CORRECT VS INCORRECT CORPUS
##For now, we will hold onto both the correct and incorrect corpuses. Later, you will only process the incorrect corpus, and the correct corpus is only used as a reference to check for recovery accuracy.
def recovery_rate(new_corpus, correct_corpus):
    wrong = 0
    for k in range(len(new_corpus)):
        if new_corpus[k] != correct_corpus[k]:
            wrong += 1
    return 1.- wrong/(len(new_corpus)+0.)
print('current recovery rate', recovery_rate(data['corpus'],data['corrupted_corpus']))

## Probability of a word mispelling
## We will use the following function to predict whether a misspelled word was actually another word. To avoid numerical issues, we make sure that the probablity is always something nonzero.
def prob_correct(word1,word2):
    SMALLNUM = 0.000001 
    if len(word1) != len(word2): return SMALLNUM
    num_wrong = np.sum(np.array([word1[k] == word2[k] for k in range(len(word1))]))
    return np.maximum(num_wrong / (len(word1) + 0.),SMALLNUM)

print ('prob not misspelling alice vs alace', prob_correct('alice','alace'))
print ('prob not misspelling alice vs earth', prob_correct('alice','earth'))
print ('prob not misspelling machinelearning vs machinedreaming', prob_correct('machinelearning','machinedreaming'))
print ('prob not misspelling machinelearning vs artificalintell', prob_correct('machinelearning','artificalintell'))

##HASHING
#all of our objects should be vectors of length V or matrices which are V x V. 
#the kth word in the vocab list is represented by the kth element of the vector, and the relationship between the i,jth words is represented in the i,jth element in the matrix.
# to easily go between the word indices and words themselves, we need to make a hash table. 
vocab_hash = {}
for k in range(len(vocab)):
    vocab_hash[vocab[k]] = k
    
#now, to access the $k$th word, we do vocab[k]. To access the index of a word, we call vocab_hash[word].




2759
current recovery rate 0.7716434266712013
prob not misspelling alice vs alace 0.8
prob not misspelling alice vs earth 1e-06
prob not misspelling machinelearning vs machinedreaming 0.6666666666666666
prob not misspelling machinelearning vs artificalintell 1e-06


In [ ]:
## FILL ME IN ##

#WORD FREQUENCY
#create an array of length V where V[k] returns the normalized frequency of word k in the entire data corpus. Do so by filling in this function.
def get_word_prob(corpus):
    vocab = np.unique(corpus)
    length = len(vocab)
    word_prob = np.zeros(length)
    frequency = Counter(corpus)
    for i,key in enumerate(vocab):
        word_prob[i] =  frequency[key]/(len(corpus) + 0.)
    
    return word_prob
 
word_prob =  get_word_prob(data['corpus'])

#report the answer of the following:
print ('prob. of "alice"', word_prob[vocab_hash['alice']])
print ('prob. of "queen"', word_prob[vocab_hash['queen']])
print ('prob. of "chapter"', word_prob[vocab_hash['chapter']])

prob. of "alice" 0.014548615047424706
prob. of "queen" 0.002569625514869818
prob. of "chapter" 0.0009069266523069947


# No Smoothing

In [ ]:
# Pr(word | prev word) 
# Using the uncorrupted corpus, accumulate the conditional transition probabilities. Do so via this formula:
# pr(word | prev) = max(# times 'prev' preceded 'word' , 1) / # times prev appears
# where again, we ensure that this number is never 0 with some small smoothing.
def get_transition_matrix(corpus):
    SMALLNUM = 0.000001 
    vocab = np.unique(corpus)
    length = len(corpus)
    # transition_matrix = np.ones((len(vocab),len(vocab)))*SMALLNUM 
    transition_matrix = np.zeros((len(vocab),len(vocab)))
    for key in range(length-1):
        i = vocab_hash[corpus[key]]
        j = vocab_hash[corpus[key+1]]
        transition_matrix[j, i] = transition_matrix[j, i] + 1  # note key+1 is the original word

    for i in range(len(vocab)):
        transition_matrix[:, i] = transition_matrix[:, i] / corpus.count(vocab[i])

    return transition_matrix

transition_matrix = get_transition_matrix(data['corpus'])
print ('prob. of "the alice"', transition_matrix[vocab_hash['alice'],vocab_hash['the']])
print ('prob. of "the queen"', transition_matrix[vocab_hash['queen'],vocab_hash['the']])
print ('prob. of "the chapter"', transition_matrix[vocab_hash['chapter'],vocab_hash['the']])
print ('prob. of "the hatter""', transition_matrix[vocab_hash['hatter'],vocab_hash['the']])

prob. of "the alice" 0.0
prob. of "the queen" 0.03968253968253968
prob. of "the chapter" 0.0
prob. of "the hatter"" 0.031135531135531136


In [ ]:
# The prior probabilities are just the word frequencies
prior = word_prob  + 0.

# Write a function that returns the emission probability of a potentially misspelled word, by comparing its probabilities against every word in the correct vocabulary

def get_emission(mword):
    return np.zeros(V)

def get_prob():
    prob = np.zeros(V)
    for i in range(V):
        prob[i] = prob_correct(mword, vocab[i])
    return prob

#find the 10 closest words to 'abice' and report them
idx = np.argsort(get_emission('abice'))[::-1]
print([vocab[j] for j in idx[:10]])


['abide', 'alice', 'above', 'voice', 'alive', 'twice', 'thick', 'dance', 'stick', 'prize']


In [ ]:
#now we reduce our attention to a small segment of the corrupted corpus
corrupt_corpus =   data['corrupted_corpus'][:1000]
correct_corpus =   data['corpus'][:1000]


In [ ]:
# encode the HMM spelling corrector. To debug, you can see the first hundred words of both the corrupted and proposed corpus, to see if spelling words got corrupted.

# report the recovery rate of the proposed (corrected) corpus.


# Forward Step
forward_probs = np.ones((len(corrupt_corpus), V))/(V+0.)

emission_mat = get_emission(corrupt_corpus[0])

for i in range(V): 
    forward_probs[0, i] = emission_mat[i]*word_prob[i]

for j in range(len(corrupt_corpus)-1):

    emission_pb = get_emission(corrupt_corpus[j+1])

    for i in range(V):
            trans = transition_matrix[i, :]

            emssion = emission_pb[i]
            forward_probs[j + 1, i] = emssion * np.sum(trans * forward_probs[j, :])

    forward_probs[j + 1, :] = forward_probs[j+1, :]/np.sum(forward_probs[j + 1, :])

# Backward Step
backward_probs = np.ones((len(corrupt_corpus), V))/(V+0.)

for i in range(len(corrupt_corpus))[::-1]:
    if i == len(corrupt_corpus) - 1:
        for j in range(V):
            backward_probs[i, j] = 1.

    else:
        emission_mat = get_emission(corrupt_corpus[i + 1])
        for j in range(V):
            trans = transition_matrix[:, j].T

            backward_probs[i, j] = np.sum(emission_mat * trans * backward_probs[i+1, :])

    backward_probs[i, :] = backward_probs[i, :]/np.sum(backward_probs[i, :])


# compute corrected corpus
proposed_corpus = copy.deepcopy(corrupt_corpus)

for i in range(len(proposed_corpus)):
    prob_z = forward_probs[i, :] * backward_probs[i, :]
    proposed_corpus[i] = vocab[np.argmax(prob_z)]

for j in range(100):
    print(proposed_corpus[j], corrupt_corpus[j])
    print('Correct world: ' + correct_corpus[j])

print(recovery_rate(corrupt_corpus, correct_corpus), recovery_rate(proposed_corpus, correct_corpus))

    

    

alices alices
Correct world:alices
adventures adventures
Correct world:adventures
in in
Correct world:in
wonderland wonderland
Correct world:wonderland
by yb
Correct world:by
lewis lewia
Correct world:lewis
carroll carroll
Correct world:carroll
the the
Correct world:the
millennium millennium
Correct world:millennium
fulcrum fulcrkm
Correct world:fulcrum
edition edition
Correct world:edition
30 30
Correct world:30
contents contents
Correct world:contents
chapter chapter
Correct world:chapter
i i
Correct world:i
dont down
Correct world:down
the tqe
Correct world:the
rabbithole raibbthole
Correct world:rabbithole
chapter chapter
Correct world:chapter
ii ii
Correct world:ii
the the
Correct world:the
pool pooo
Correct world:pool
of of
Correct world:of
tears aetrs
Correct world:tears
chapter chapter
Correct world:chapter
iii dii
Correct world:iii
a a
Correct world:a
caucusrace caucusrace
Correct world:caucusrace
and and
Correct world:and
a a
Correct world:a
long long
Correct world:long
tale 